In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -q peft transformers datasets accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.2 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 90.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4


In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/MentalQA/Train_Dev.tsv", sep="\t")  # or .csv
df.head()


,question,answer,final_QT,final_AS
0,هل يعتبر الخوف من عدم الإنجاب مستقبلاً حالة عا...,سد تى نعم ان هناك العديد من القلق النفسى المرت...,"['A', 'D']","['1', '2']"
1,من سنه تقريبا و انا أذي نفسي ب اكثر من طريقة و...,يجب العرض علي طبيب اختصاصي امراض نفسية سوف يقو...,"['B', 'E']",['2']
2,السلام عليكم مشكلتي تقتصر على تكرار كلمة معينة...,الوسواس القهري هو من الامراض الشاءعة ويكون في ...,"['A', 'E']","['1', '2']"
3,اكتئاب وفوبيا من المجتمع وانعزال وانطوائية وتع...,العلاج النفسي المعرفي السلوكي يعطي نتائج جيدة ...,"['B', 'D']",['1']
4,هل الإحساس بقرب الاجل و الخوف من الموت و الاحل...,نعم بالاضافه للكثير من الاعراض الاخرى الطبيب ا...,"['A', 'B', 'D']","['1', '2']"


In [ ]:
df["prompt"] = "السؤال: " + df["question"] + "\nالجواب: "
df["output"] = df["answer"]


In [ ]:
from datasets import Dataset

dataset = Dataset.from_pandas(df[["prompt", "output"]])
dataset = dataset.train_test_split(test_size=0.15)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load clean model
model_name = "UBC-NLP/araT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

# Ensure pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("UBC-NLP/araT5-base").to("cuda")


In [ ]:
def tokenize(example):
    inputs = tokenizer(
        example["prompt"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    targets = tokenizer(
        example["output"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )
    inputs["labels"] = targets["input_ids"]
    return inputs


In [ ]:
tokenized_dataset = dataset.map(tokenize, batched=True)


Map:   0%|          | 0/297 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q", "v"],  # For T5-based models
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 884,736 || all params: 283,655,424 || trainable%: 0.3119


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./arat5-qlora",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    learning_rate=2e-4,
    save_strategy="epoch",  # Still save at the end of each epoch
    fp16=True,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],  # Still evaluated after training if you want
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-12-2801130562.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()


In [ ]:
def generate_answer(question):
    prompt = f"السؤال: {question}\nالجواب:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512  # add fixed length
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        top_p=0.9,
        temperature=0.8,
        pad_token_id=tokenizer.pad_token_id  # critical fix
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
# Ensure padding token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Explicitly assign it to model
model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
print(generate_answer("أشعر بالتعب وعدم القدرة على النوم، ماذا أفعل؟"))